# Fetching and Summarizing NWM Gridded Data

## Overview
In this guide we'll demonstrate fetching National Water Model (NWM) operational and retrospective gridded data from cloud storage. This example makes use of a pre-generated Evaluation dataset stored in TEEHR's examples data module.

**Note**: For demonstration purposes several cells below are shown in markdown form. If you want to download this notebook and run them yourself, you will need to convert them to code cells.

For a refresher on loading location and location crosswalk data into a new Evaluation refer back to the [Loading Local Data](https://rtiinternational.github.io/teehr/user_guide/notebooks/02_loading_local_data.html) and [Setting-up a Simple Example](https://rtiinternational.github.io/teehr/user_guide/notebooks/04_setup_simple_example.html) user guide pages.

### Set up the example Evaluation

In [ ]:
from datetime import datetime
from pathlib import Path
import shutil
import hvplot.pandas  # noqa
import pandas as pd

from teehr.examples.setup_nwm_grid_example import setup_nwm_example
import teehr
from teehr.evaluation.utils import print_tree

# Tell Bokeh to output plots in the notebook
from bokeh.io import output_notebook
output_notebook()

We'll start with the Evaluation from the previous User Guide, which contains observed (USGS) and simulation (NWM) streamflow for a gage at Radford, VA during hurricane Helene.

In [ ]:
# Define the directory where the Evaluation will be created.
test_eval_dir = Path(Path().home(), "temp", "11_fetch_gridded_nwm_data")

# Setup the example evaluation using data from the TEEHR repository.
shutil.rmtree(test_eval_dir, ignore_errors=True)
setup_nwm_example(tmpdir=test_eval_dir)

# Initialize the evaluation.
ev = teehr.Evaluation(dir_path=test_eval_dir)

In [ ]:
ev.locations.to_geopandas()

We can add polygons to summarize rainfall to, to see the impact of the forcing data on the NWM streamflow prediction

In [ ]:
location_data_path = Path(test_eval_dir, "three_huc10s_radford.parquet")

ev.locations.load_spatial(
    location_data_path,
    field_mapping={
        "huc10": "id"
    },
    location_id_prefix="wbd",
    write_mode="append"  # this is the default
)

Now we have the USGS gage point, and three WBD HUC10 polygons in the locations table.

In [ ]:
gdf = ev.locations.to_geopandas()
gdf

In [ ]:
gdf[gdf.id != "usgs-03171000"].hvplot(geo=True, tiles="OSM", alpha=0.5) * \
    gdf[gdf.id == "usgs-03171000"].hvplot(geo=True, color="black", size=50)

Now we'll fetch NWM gridded analysis and assimilation rainfall data and calculate mean areal precipitation (MAP) for each of the HUC10 watersheds. We'll consider this "observed" primary timeseries. 

The location ID prefix is used to filter the locations table for WBD polygons during fetching. The calculated MAP timeseries will be appended (the default) to the `primary_timeseries` table along with the streamflow data (the primary timeseries table will contain both streamflow and rainfall).

```python
# To run this cell locally first convert it to a python cell
ev.fetch.nwm_operational_grids(
    nwm_configuration="forcing_analysis_assim",
    output_type="forcing",
    variable_name="RAINRATE",
    start_date=datetime(2024, 9, 26),
    end_date=datetime(2024, 9, 26),
    nwm_version="nwm30",
    prioritize_analysis_valid_time=True,
    t_minus_hours=[0],
    calculate_zonal_weights=True,
    location_id_prefix="wbd",
    timeseries_type="primary"  # To be considered "observations". This is the default.
)
```

In [ ]:
primary_df = ev.primary_timeseries.to_pandas()
primary_df.head()

Let's take a look at the `cache` directory. After fetching the NWM analysis forcing, we have a directory for the weights file.

In [ ]:
print_tree(Path(ev.cache_dir, "fetching", "weights"))

The weights file contains the fractional coverage for each pixel (row/col pair) that intersects with each polygon location (WBD HUC10 watershed).

In [ ]:
cached_weights_filepath = Path(
        ev.cache_dir,
        "fetching",
        "weights",
        "nwm30_forcing_analysis_assim",
        "nwm30_forcing_analysis_assim_pixel_weights.parquet"
)
pd.read_parquet(cached_weights_filepath)

Now we can fetch the NWM forecast rainfall and calculate MAP, to compare it to the "observed" rainfall. First we need to create and import the `location_crosswalks` table, which maps the secondary locations ("model") to the primary ("observations").

In [ ]:
ev.locations.to_pandas()["id"].unique()

In [ ]:
xwalk_df = pd.DataFrame(
    {
        "primary_location_id": [
            "wbd-0505000115",
            "wbd-0505000117",
            "wbd-0505000118"
        ],
        "secondary_location_id": [
            "forecast-0505000115",
            "forecast-0505000117",
            "forecast-0505000118"
        ]
    }
)
temp_xwalk_path = Path(ev.cache_dir, "forcing_xwalk.parquet")
xwalk_df.to_parquet(temp_xwalk_path)

Now we can load it into the Evaluation, appending to the existing table.

In [ ]:
ev.location_crosswalks.load_parquet(temp_xwalk_path)
ev.location_crosswalks.to_pandas()

Now we can fetch the NWM forecast rainfall, which we'll consider the "secondary" timeseries. Let's grab a single short range rainfall forecast

```python
# To run this cell locally first convert it to a python cell
ev.fetch.nwm_operational_grids(
    nwm_configuration="forcing_short_range",
    output_type="forcing",
    variable_name="RAINRATE",
    start_date=datetime(2024, 9, 26),
    end_date=datetime(2024, 9, 26),
    nwm_version="nwm30",
    location_id_prefix="forecast",
    calculate_zonal_weights=False,  # re-use the weights file in the cache
    zonal_weights_filepath=cached_weights_filepath,
    starting_z_hour=1,
    ending_z_hour=1,
    timeseries_type="secondary"  # now considered forecast
)
```

And our `secondary_timeseries` table consists of NWM medium range streamflow forecasts (member 1), plus NWM short range rainfall forecast summarized to the three WBD HUC10 watersheds as mean areal precipitation (MAP).

In [ ]:
secondary_df = ev.secondary_timeseries.to_pandas()
secondary_df.head()

And our `primary_timeseries` table consists of USGS streamflow data at the Radford gage, plus NWM analysis rainfall summarized to the three WBD HUC10 watersheds as mean areal precipitation (MAP).

In [ ]:
primary_df = ev.primary_timeseries.to_pandas()
primary_df.head()

Now we can create the `joined_timeseries` table allowing for efficient metric calculation and further exploration.

In [ ]:
ev.joined_timeseries.create()

joined_df = ev.joined_timeseries.to_pandas()
joined_df.head()

In [ ]:
joined_df.configuration_name.unique()

In [ ]:
# We'll use this helper function for plotting timeseries.
def plot_timeseries(
    configuration_name: str,
    secondary_by: list[str],
    title: str
):
    sim_plot = joined_df[joined_df["configuration_name"] == configuration_name].\
        hvplot(
            x="value_time",
            y="secondary_value",
            by=secondary_by,
            legend=False,
            )
    obs_plot = joined_df[joined_df["configuration_name"] == configuration_name].\
        hvplot(
            x="value_time",
            y="primary_value",
            by=["primary_location_id", "configuration_name"],
            legend=False,
            color="black"
            )
    return (sim_plot * obs_plot).options(
                width=700,
                height=400,
                show_grid=True,
                title=title
            )

We can plot the primary (USGS) vs. secondary (NWM) streamflow and notice that this particular NWM forecast underestimated the flow.

In [ ]:
plot_timeseries(
    configuration_name="nwm30_medium_range",
    secondary_by=["reference_time"],
    title="Observed streamflow (black) and forecast streamflow at the Radford gage"
)

In plotting the primary vs. secondary rainfall (MAP) in each watershed, we can see the this NWM forecast underestimated the observed rainfall. This is likely one reason why the NWM streamflow forecasts also underestimated the flow.

In [ ]:
plot_timeseries(
    configuration_name="nwm30_forcing_short_range",
    secondary_by=["primary_location_id"],
    title="Observed MAP (black) and forecast MAP at the three HUC10s"
)

Now we can also calculate performance metrics for the streamflow and rainfall for this event.

In [ ]:
bias = teehr.DeterministicMetrics.RelativeBias()
kge = teehr.DeterministicMetrics.KlingGuptaEfficiency()

ev.metrics.query(
    include_metrics=[bias, kge],
    group_by=["primary_location_id", "configuration_name"],
    order_by=["primary_location_id", "configuration_name"],
).to_pandas()

In [ ]:
ev.spark.stop()